In [2]:
import pandas as pd
import json
import csv
import re
import tweepy
from tweepy import OAuthHandler
import time

In [3]:
df_archive = pd.read_csv('twitter-archive-enhanced-2.csv')
df_image = pd.read_csv('image-predictions-3.tsv',  delimiter='\t')

In [4]:
df_archive.head()
#df.info()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
consumer_key = 'jbL7N3K4eXgD1zhDfeS2TcnFr'
consumer_secret = 'XeIpJqZ4i8WAolHVCNBqGtILZKY1J0TVJw8GkpW9jDRthls1qn'
access_token = '495295426-3dc2rEDWSI8AX8NxYCpIRV9tk83EwocP7RzbraDC'
access_token_secret = 'zsulnNzoCcHRURBOkfVkb1tLcJPfYinSwfJBpn9Nam1FV'


# search_favorites(consumer_key, consumer_secret, access_token, access_token_secret):
#create authentication for accessing twitter
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#initialize Tweepy API
api = tweepy.API(auth, wait_on_rate_limit =True, wait_on_rate_limit_notify = True)
tweet_ids = df_archive.tweet_id



In [6]:
count = 0
fails_dict = {}
#save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    
    #loop through each tweet id and write to outfile
    for tweet_id in tweet_ids:
        count += 1        
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n')
            
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass


Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail


Rate limit reached. Sleeping for: 596


Fail
Fail


Rate limit reached. Sleeping for: 628


In [10]:
df_1 = pd.DataFrame(columns=['tweet_id', 'retweet_count', 'favorite_count'])
with open('tweet_json.txt') as f:
    for line in f:
        status  = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        df_1 = df_1.append(pd.DataFrame([[tweet_id, retweet_count, favorite_count]],
                                        columns=['tweet_id', 'retweet_count', 'favorite_count']))
df_1 = df_1.reset_index(drop=True)


In [17]:
df_1.tweet_id = df_1.tweet_id.astype('str')
df_archive.tweet_id = df_archive.tweet_id.astype('str')
#assess
#df_merged2.query('rating_numerator < 10 or rating_numerator >16')['text']
#df_merged2.name.value_counts()


Merge data frames

In [18]:
df_merged = df_archive.merge(df_1, how= 'inner', on = 'tweet_id')
#df_archive.info()
#df_1.info()

def: remove rows present in retweet status
remove unnecessary rows
remove retweet status columns
remove denomenator

In [19]:
#df1 = df1[df1['retweeted_status_id'].isna()]
df_merged2 = df_merged[df_merged['retweeted_status_id'].isna()]
df_merged2 = df_merged[df_merged['in_reply_to_status_id'].isna()]

df_merged2.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'rating_denominator', 'in_reply_to_status_id', 'in_reply_to_user_id'],  axis =1, inplace = True )

C:\Users\Summer\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


combine stages

In [20]:
#df_merged2['stage'] = df_merged2.doggo + df_merged2.floofer + df_merged2.pupper + df_merged2.puppo
#df_merged2.doggo.value_counts()


remove rows without photos

In [21]:
#df_merged2.info()
#df_merged['image_link'] = df_merged.text.str.extract('http://')
df_merged3 = df_merged2[df_merged2.text.str.contains("https://")]

Names 869988702071779329 = quite
a 
an 
the

In [22]:
df_merged3.loc[df_merged3['name'] == "quite", 'name'] = None
df_merged3.loc[df_merged3['name'] == "the", 'name'] = None
df_merged3.loc[df_merged3['name'] == "an", 'name'] = None
df_merged3.loc[df_merged3['name'] == "an", 'name'] = None
df_merged3.loc[df_merged3['name'] == "just", 'name'] = None



C:\Users\Summer\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


combine stage columns

In [23]:

df_merged3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2227 entries, 0 to 2336
Data columns (total 13 columns):
tweet_id            2227 non-null object
timestamp           2227 non-null object
source              2227 non-null object
text                2227 non-null object
expanded_urls       2227 non-null object
rating_numerator    2227 non-null int64
name                2205 non-null object
doggo               2227 non-null object
floofer             2227 non-null object
pupper              2227 non-null object
puppo               2227 non-null object
retweet_count       2227 non-null object
favorite_count      2227 non-null object
dtypes: int64(1), object(12)
memory usage: 243.6+ KB


In [40]:
df_merged3['stage'] = df_merged3.text.str.findall(r'(doggo|floofer|pupper|puppo)')
#df_merged3['stage']= df_merged3['floofer'] + df_merged3['doggo'] + df_merged3['pupper'] + df_merged3['puppo']


C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
df_merged3['stage'] = df_merged3['stage'].str.join(',')


C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
df4 = df_merged3.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis =1)

In [43]:


df4['stage'].replace('pupper,pupper', 'pupper', inplace =True) 
df4['stage'].replace('pupper,doggo,doggo', 'pupper,doggo', inplace = True) 
df4['stage'].replace('pupper,pupper,pupper', 'pupper', inplace = True) 

df4.stage.value_counts()

                1849
pupper           251
doggo             81
puppo             31
doggo,pupper       6
floofer            4
pupper,doggo       3
puppo,doggo        2
Name: stage, dtype: int64

In [30]:
import datetime as dt

df4.timestamp = pd.to_datetime(df4['timestamp'])

df4['day_of_week'] = df4['timestamp'].dt.dayofweek

In [31]:
df4.stage

0             
1             
2             
3             
4             
5             
6             
7             
8             
9        doggo
10            
11            
12       puppo
13            
14       puppo
15            
16            
17            
18            
19            
20            
21            
22            
23            
24            
25            
26            
27            
28      pupper
30            
         ...  
2307          
2308          
2309          
2310          
2311          
2312          
2313          
2314          
2315          
2316          
2317          
2318          
2319          
2320          
2321          
2322          
2323          
2324          
2325          
2326          
2327          
2328          
2329          
2330          
2331          
2332          
2333          
2334          
2335          
2336          
Name: stage, Length: 2227, dtype: object

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [44]:
df_1 = df_image.query('p1_dog == True')
df_2 = df_image.query('p1_dog != True & p2_dog == True')
df_3 = df_image.query('p1_dog != True & p2_dog != True & p3_dog == True')

df_1['breed'] = df_1['p1']
df_2['breed'] = df_2['p2']
df_3['breed'] = df_3['p3']
df_3.breed
len(df_1) + len(df_2) + len(df_3)
len(df_image)

C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

2075

In [45]:
image_clean = pd.concat([df_1, df_2, df_3], axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
image_clean = image_clean[['tweet_id','breed']]

In [46]:
#image['tweet_id']
image_clean = image_clean.astype(str, copy=True)
df4.merge(image_clean, how = 'left', on = 'tweet_id')
df4.info()
df4.tweet_id.duplicated().any()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2227 entries, 0 to 2336
Data columns (total 10 columns):
tweet_id            2227 non-null object
timestamp           2227 non-null object
source              2227 non-null object
text                2227 non-null object
expanded_urls       2227 non-null object
rating_numerator    2227 non-null int64
name                2205 non-null object
retweet_count       2227 non-null object
favorite_count      2227 non-null object
stage               2227 non-null object
dtypes: int64(1), object(9)
memory usage: 191.4+ KB


False

0             
1             
2             
3             
4             
5             
6             
7             
8             
9        doggo
10            
11            
12       puppo
13            
14       puppo
15            
16            
17            
18            
19            
20            
21            
22            
23            
24            
25            
26            
27            
28      pupper
30            
         ...  
2307          
2308          
2309          
2310          
2311          
2312          
2313          
2314          
2315          
2316          
2317          
2318          
2319          
2320          
2321          
2322          
2323          
2324          
2325          
2326          
2327          
2328          
2329          
2330          
2331          
2332          
2333          
2334          
2335          
2336          
Name: stage, Length: 2227, dtype: object

Test:

In [36]:
df_image.query('p1_dog != True & p2_dog != True & p2_dog != True').shape

(388, 12)

Re extract data to test its accuracy

In [187]:
df_test = pd.DataFrame(columns=['tweet_id', 'text'])
with open('tweet_json.txt') as f:
    for line in f:
        status  = json.loads(line)
        tweet_id = status['id_str']
        text = status['full_text']
        df_test = df_test.append(pd.DataFrame([[tweet_id, text]],
                                        columns=['tweet_id', 'text']))
df_test = df_test.reset_index(drop=True)


In [198]:
df_test = df_test[df_test['tweet_id'].isin(df4['tweet_id'])]
                  
df_test['rating'] = df_test.text.str.findall(r'(\d+/10)')
a = df_test['rating'].str.extract(r'(/+){2}')
#df_test.rating.value_counts()
#df4.rating_numerator.value_counts()
#a = df_test['rating'].str.findall(r'.{6,}$')
#a.value_counts()
#^[a-zA-Z]{3,7}$
df_test.to_csv('df_test.csv')


In [199]:
#df_test.rating.str.findall(r'(\d+/10){2}')
#df_test.rating.count('/')
#str(count['e'])
#len(re.findall("/", df_test['rating'])) 
#df_test['rating'].str.findall('/') 
df_test = df_test[(df_test['rating'].str.len() < 2) & (df_test['rating'].str.len() > 0)]

df_test.rating = df_test.rating.str.get(0)

In [200]:
df_test.rating.str.split('/', expand = True)
df_test.rating  = df_test.rating.str.split('/', expand = True)[0]

In [203]:
df_test.rating.value_counts()
len(df4)
len(df_test)

2192


666287406224695296
749981277374128128 1776
670842764863651840 420
75        2 
832215909146226688 should be 9.75

0         1
26        1 look at 683773439333797890
27  

In [207]:
df_test.to_csv('df_test.csv')
df_test.shape

(2192, 3)

In [39]:

731156023742988288


731156023742988288

Series([], Name: stage, dtype: int64)

In [ ]:
change variable types
tweetid should be a string
stage characters should be categories 

fix the names column
meet, this is, 

Delete the rating denomenator

images: extract type of dog and merge with bigger table


In [ ]:
import matplotlib 
import seaborn
df4['stage'].plot()

plt.style.use('seaborn-darkgrid')
oil_trans.mean(axis = 1).plot()
plt.title("Global Oil Production")
plt.ylabel('Tonnes Oil Equvalent')

y1 = gdp_trans.mean(axis = 1)
y2 = gdp_oil_trans.mean(axis = 1)
plt.title("GDP/capita")
plt.ylabel('International dollars')
all_countries = plt.plot(y1)
oil_producing_countries = plt.plot(y2)
plt.legend( ['All Countries', 'Oil Producing Countries'])

oil_gdp = gdp_oil.mean().mean()
global_gdp = gdp.mean().mean()
height = [global_gdp, oil_gdp]
bars = ('All Countries', 'Oil Countries')
y_pos = np.arange(len(bars))
plt.bar(y_pos,height, width = .4, color = ['red', 'blue'])
plt.ylabel('International Dollars')
plt.xticks(y_pos, bars)
plt.title('Average GDP/capita')

oil_level = agg_df.groupby(['oil_prod_level']).mean()
oil_plot = oil_level.plot(y = 'mean_gini', kind = 'bar', yticks = [10,20,30, 40, 50], color = ['blue', 'green', 'purple', 'red'], legend = False)
oil_plot.set_title('GINI Index by amount of oil production')
oil_plot.set_ylabel('GINI Index(%)')

agg_df['gdp_level'] = agg_df['mean_gdp'].apply(gdp_level)

oil_gdp_level = agg_df.groupby(['gdp_level', 'oil_prod_level']).mean()
oil_gdp_plot = oil_gdp_level.unstack().plot(y = 'mean_gini', kind='bar')
oil_gdp_plot.set_ylabel('GINI Index(%)')
oil_gdp_plot.set_title('GINI index grouped by Oil Production and GDP')